<a href="https://colab.research.google.com/github/EllingtonKirby/Workbooks/blob/master/formant_extraction_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchaudio

import torchaudio
from torchaudio import datasets
from IPython.display import Audio

     |████████████████████████████████| 1.9 MB 31.1 MB/s 


In [ ]:
data = datasets.CMUARCTIC(root=".", download=True, folder_in_archive="ARCTIC", url="aew")

In [ ]:
n = 0
(waveform, sample_rate, utterance, utterance_id) = data.__getitem__(10)

wave_dir = '.'
wave_filename = f'{wave_dir}/{utterance_id}.wav'
torchaudio.save(wave_filename, waveform, sample_rate, bits_per_sample=16)

In [ ]:
audio = Audio(data=waveform, rate=sample_rate)
audio

In [ ]:
pip install praat-parselmouth

     |████████████████████████████████| 10.7 MB 19.4 MB/s 


In [ ]:
#!/usr/bin/env python3
import glob
import numpy as np
import pandas as pd
import parselmouth 
import statistics


from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# This function measures formants using Formant Position formula
def measureFormants(sound, wave_file, f0min,f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median


In [ ]:
example_sound = parselmouth.Sound(wave_filename)

In [ ]:
measureFormants(example_sound, wave_filename, 75, 300)

(465.5668566371183,
 1649.105877919037,
 2579.709416888426,
 3488.928334809783,
 399.9728952046444,
 1619.0291479279767,
 2579.362076210593,
 3518.315964136162)

In [ ]:
# This function measures formants using Formant Position formula
def measureFormantLists(wave_file, f0min,f0max):
    sound = parselmouth.Sound(wave_file) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    formant_list = []
    amp_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        # print(f'time is: ${t}')
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        amp = call(sound, "Get value at time", 1, t, 'sinc70')

        formant_list.append((f1, f2, t))
        amp_list.append(amp)
    
    # for index, amp in enumerate(amp_list):
    #   print(f'Amplitude: {amp} for formant {f1_list[index]}')
    
    return formant_list

In [ ]:
f1_f2_timeseries = measureFormantLists(wave_filename, 75, 300)

In [ ]:
formant_dict_full = {240.0:'i', 235.0:'y', 390.0:'e', 370.0:'ø', 610.0:'ɛ', 585.0:'œ', 850.0:'a', 820.0:'ɶ', 750.0:'ɑ', 700.0:'ɒ', 600.0:'ʌ', 500.0:'ɔ',460.0:'ɤ', 360.0:'o', 300.0:'ɯ', 250.0:'u'}

In [ ]:
formant_dict_reduced = {
    (850, 1610):'a', 
    (390, 2300):'e', 
    (240, 2400):'i',
    (360,  640):'o', 
    (250,  595):'u'
}

In [ ]:
f1_f2_dict = {
    (240, 2400):'i',
    (235, 2100):'y',
    (390, 2300):'e', 
    (370, 1900):'ø', 
    (610, 1900):'ɛ', 
    (585, 1710):'œ', 
    (850, 1610):'a', 
    (820, 1530):'ɶ', 
    (750, 940): 'ɑ', 
    (700, 760): 'ɒ', 
    (600, 1170):'ʌ', 
    (500, 700): 'ɔ',
    (460, 1310):'ɤ', 
    (360, 640): 'o', 
    (300, 1390):'ɯ', 
    (250, 595): 'u'
}

In [ ]:
def extract_ipa_from_formants(formant_timeseries, formant_dict):
  retval = []
  keys = list(formant_dict.keys())
  values = list(formant_dict.values())
  np_array = np.asarray(keys)
  for f1, f2, time in formant_timeseries:
    normed_output = []
    for formant_baseline in keys:
      difference = np.linalg.norm(np.subtract(formant_baseline, [f1, f2]))
      normed_output.append(difference)
    idx = np.argmin(normed_output)
    retval.append((values[idx], time))
  return retval

In [ ]:
ipa_from_f1 = extract_ipa_from_formants(f1_f2_timeseries, formant_dict_reduced)


In [ ]:
ipa_counts_from_f1 = np.unique(ipa_from_f1, return_counts=True)

In [ ]:
from IPython.display import Image

image = Image("/content/drive/MyDrive/Visemes/illustration_voicemoji_lips_a_96.png"); image

In [ ]:
def generate_ipa_boundaries(ipa_timeseries):
  output = []
  for index, value in enumerate(ipa_timeseries):
    ipa, time = value[0], value[1]
    if (len(output) == 0):
      output.append((ipa, time))
    else:
      current_symbol, last_time = output.pop()
      if (current_symbol == ipa):
        output.append((current_symbol, last_time + (time - total_time)))
      else:
        output.append((current_symbol, last_time))
        output.append(('X', .005))
        output.append((ipa, (time - total_time)))
    total_time = time
  return output


In [ ]:
ipa_boundaries = generate_ipa_boundaries(ipa_from_f1)
ipa_boundaries

In [ ]:
ipa_to_png = {
    'a':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_a_96.png",
    'e':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_e_96.png",
    'i':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_e_96.png",
    'o':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_o_96.png",
    'u':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_wr_96.png",
    'y':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_wr_96.png",
    'ø':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_ln_96.png",
    'ɛ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_ts_96.png",
    'œ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_e_96.png",
    'ɶ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_a_96.png",
    'ɑ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_a_96.png",
    'ɒ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_o_96.png",
    'ʌ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_o_96.png",
    'ɔ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_uq_96.png",
    'ɤ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_wr_96.png",
    'ɯ':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_uq_96.png",
    'X':"/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_smile_96.png"
}

In [ ]:
images_and_durations = [(ipa_to_png[symbol[0]], symbol[1]) for symbol in ipa_boundaries]

In [ ]:
pip install moviepy

In [ ]:
from moviepy.editor import *

clips = [ImageClip(m[0]).set_duration(m[1]) for m in images_and_durations]

concat_clip = concatenate_videoclips(clips, method="compose")
concat_clip = concat_clip.set_audio(AudioFileClip(wave_filename))
concat_clip.write_videofile("test.mp4", fps=24)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3407872/45929032 bytes (7.4%)6750208/45929032 bytes (14.7%)10223616/45929032 bytes (22.3%)13647872/45929032 bytes (29.7%)17285120/45929032 bytes (37.6%)21012480/45929032 bytes (45.7%)24707072/45929032 bytes (53.8%)28426240/45929032 bytes (61.9%)32047104/45929032 bytes (69.8%)35741696/45929032 bytes (77.8%)39411712/45929032 bytes (85.8%)43122688/45929032 bytes (93.9%)

100%|██████████| 86/86 [00:00<00:00, 961.62it/s]

[MoviePy] Done.
[MoviePy] Writing video test.mp4



100%|██████████| 88/88 [00:00<00:00, 166.35it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: test.mp4 



In [ ]:
concat_clip.ipython_display(width=200, fps=24)

100%|██████████| 88/88 [00:00<00:00, 177.54it/s]


Lets put it all together

In [ ]:
def generate_lip_sync(filename, formant_dictionary, phoneme_png_dictionary):
  formants = measureFormantLists(wave_filename, 75, 300)
  ipa_list = extract_ipa_from_formants(formants, formant_dictionary)
  unique_ipa_boundaries = generate_ipa_boundaries(ipa_list)
  return [(phoneme_png_dictionary[symbol[0]], symbol[1]) for symbol in unique_ipa_boundaries]

In [ ]:
(waveform, sample_rate, utterance, utterance_id) = data.__getitem__(15)
wave_dir = '.'
wave_filename = f'{wave_dir}/{utterance_id}.wav'
torchaudio.save(wave_filename, waveform, sample_rate, bits_per_sample=16)

In [ ]:
%%time
ipa_sequence = generate_lip_sync(wave_filename, f1_f2_dict, ipa_to_png)

CPU times: user 788 ms, sys: 59.6 ms, total: 848 ms
Wall time: 702 ms


In [ ]:
def generate_video_from_ipa_sequence(images, audio):
  clips = [ImageClip(m[0]).set_duration(m[1]) for m in images]

  concat_clip = concatenate_videoclips(clips, method="compose")
  audio_clip = AudioFileClip(audio)
  concat_clip = concat_clip.set_audio(audio_clip)
  concat_clip.write_videofile("test.mp4", fps=24)
  print(f'\n audio length: {audio_clip.duration}')
  print(f'video length: {concat_clip.duration}')
  return concat_clip

In [ ]:
(waveform, sample_rate, utterance, utterance_id) = data.__getitem__(2)
wave_dir = '.'
wave_filename = f'{wave_dir}/{utterance_id}.wav'
torchaudio.save(wave_filename, waveform, sample_rate, bits_per_sample=16)
ipa_sequence = generate_lip_sync(wave_filename, f1_f2_dict, ipa_to_png)
print(ipa_sequence[0:20])
generate_video_from_ipa_sequence(ipa_sequence, wave_filename).ipython_display(width=200, fps=24)

[('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_uq_96.png', 0.2114145878798721), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_smile_96.png', 0.005), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_o_96.png', 0.0260258427512971), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_smile_96.png', 0.005), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_wr_96.png', 0.026326968108146942), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_smile_96.png', 0.005), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_uq_96.png', 0.05346104953654596), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_smile_96.png', 0.005), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_lips_ts_96.png', 0.027210727671685697), ('/content/drive/MyDrive/Visemes/existing/illustration_voicemoji_smile_96.png', 0.005), ('/content/drive/MyDrive/Visemes/existin

100%|██████████| 79/79 [00:00<00:00, 1019.13it/s]

[MoviePy] Done.
[MoviePy] Writing video test.mp4



100%|██████████| 90/90 [00:01<00:00, 80.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test.mp4 



  0%|          | 0/90 [00:00<?, ?it/s]


 audio length: 3.54
video length: 3.711285906898772


100%|██████████| 90/90 [00:01<00:00, 87.38it/s]
